# Задание для проекта Reboot


1.	Выгрузить описания открытых вакансий с любой из площадок (hh.ru, rabota.sber.ru, indeed.com) или воспользоваться готовой выгрузкой ... 

2.	<a href="https://python-school.ru/nlp-text-preprocessing/">Предообработать</a> текст описаний вакансий.
    * очистить текст -  убрать лишние символы и html-тэги
    * привести к одному регистру
    * токенизировать - разбить текст на слова
    * удалить часто встречающиеся слова и стоп-слова
    * <a href="https://pymorphy2.readthedocs.io/en/stable/"> нормировать  </a>(красивые машины -> красивый машина) <br>
    подробнее про обработку естественного языка (NLP) можно прочитать <a href="https://neerc.ifmo.ru/wiki/index.php?title=Обработка_естественного_языка">здесь<a>)
3.	Взять любой текст, описывающий опыт из резюме и аналогично предобработать.
4.	Посчитать векторное представление описаний вакансий и опыта из резюме.
    * для того чтобы обрабатывать текстовые данные с помощью методов машинного обучения текст преобразуют в векторное представление или эмбеденги. 
    * для того чтобы преобразовать слова или предложения в эмбедениг используют разные алгоритмы (<a href="https://habr.com/ru/company/Voximplant/blog/446738/" > TF-IDF</a>, Word2Vec, <a href="https://fasttext.cc/docs/en/support.html"> Fasttext</a>, Bert и другие)
    
5.	Посчитать косинусную меру сходства для опыта и описаний вакансий. Подробнее о об этом  <a href="https://ru.wikipedia.org/wiki/Векторная_модель#.D0.9A.D0.BE.D1.81.D0.B8.D0.BD.D1.83.D1.81.D0.BD.D0.BE.D0.B5_.D1.81.D1.85.D0.BE.D0.B4.D1.81.D1.82.D0.B2.D0.BE"> здесь</a> и <a href="http://zabaykin.ru/?p=737"> здесь</a> 
6.	Ранжировать вакансии по мере их сходства с опытом и вывести Топ-10 наиболее релевантных вакансий
7.	Подготовить скрипт принимающий на вход текстовое описание опыта и выводящий Топ-10 наиболее релевантных вакансий из всех или 100 случайно выбранных из выгрузки


8. Проверка будет проходить в формате api, который принимает на вход строку текста, а в ответ возвращает топ-10 вакансий. Пример в приложенном скрипте сделан с помощью фреймворка  <a href="https://fastapi.tiangolo.com/">FastAPI</a>. 
* Проверить результат работы можно запустив сервис с помощью команды <b> uvicorn main:app --reload </b> и затем отправив соответствующий запрос либо с помощью встроенного сервиса http://127.0.0.1:8000/docs, либо с помощью Postman
* Шаблон для API указан в приложении


## 1.Выгружаем описания вакансий

In [3]:
import pandas as pd
import numpy as np
import csv

In [5]:
csv.field_size_limit(100000000)

131072

In [15]:
vacs = pd.read_csv('vacancy.csv')

In [16]:
vacs.shape


(4002, 10)

##  2.Предобрабатываем текст

In [21]:
vacs[:1]

,id,created_at,updated_at,vacid,vactitle,vacdescription,vacdate,vacstatus,vaclink,vachtml
0,383479,2021-03-14 09:39:21.854462,2021-03-14 09:39:21.854584,1474941,Senior java-разработчик (в команду Онбординга),"Работа у нас — это:команда профессионалов, го...",2021-03-15,new,https://my.sbertalents.ru/#/job-requisition/14...,"{""title"": ""Senior java-\u0440\u0430\u0437\u044..."


In [23]:
import re

In [65]:
# функция удаления пунктуации и цифр
def remove_trash(x): 
    desc = list(x)
    description_result = []
    for i in desc:
        i = str(i)
        y = re.sub(r'[^А-Яа-яЁё ]+', ' ', i)
        description_result.append(y)
    return description_result

vacs['vacdesc_clean'] = remove_trash(vacs['vacdescription'])

In [71]:
vacs['vacdesc_clean'][8]

' Условия    работа в крупнейшем банке России  трудоустройство согласно ТК РФ  регулярное корпоративное обучение  ДМС  страхование от несчастных случаев и тяжелых заболеваний  материальная помощь и социальная поддержка  корпоративная пенсионная программа  льготные условия кредитования  яркая и насыщенная корпоративная жизнь  Обязанности    контроль и проверка работоспособности систем безопасности  в т ч  автоматизированных  на закрепленных объектах   ОПС  контроль доступа  телевизионная система наблюдения  проведение проверочных мероприятий по своему направлению согласно инструкциям  в том числе в отношении кандидатов  осуществление мер по защите объекта  жизни и здоровья сотрудников и клиентов при обнаружении инцидентов  анализ информации  полученной в результате проверок  формирование первичных выводов  формирование справок  отчётности и заключений по своему направлению для руководителей  подготовка материалов для составления ТЗ по разработке проектно сметной документации по конкретн

In [25]:
!pip install pymorphy2

  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13709 sha256=7c38427b34fdec53af152991926ccca612b3711752bedb223fb9d400e3a25bb6
  Stored in directory: c:\users\zhenya\appdata\local\pip\cache\wheels\56\ea\58\ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt


In [26]:
import pymorphy2

In [99]:
morph = pymorphy2.MorphAnalyzer()

# функция лемматизации
def lemmatize(row):
    t = []
    text = row['vacdesc_clean']
    for word in text.split():
        if len(word)<=3:
            continue
        p = morph.parse(word)[0]
        t.append(p.normal_form)
    return " ".join(t)


vacs['vacdesc_morphed'] = vacs.apply(lemmatize, axis = 1)

KeyboardInterrupt: 

In [75]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [73]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ZHENYA\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [79]:
vacs['vacdesc_morphed']

0       работа команда профессионал готовый поддержать...
1       искать дизайнер наш дружный команда сбербанк н...
2       условие официальный трудоустройство дневный ра...
3       блок риск стартовать программа миграция фабрик...
4       работа работа крупный банк россия команда проф...
                              ...                        
3997    система статический динамический анализ исходн...
3998    аналитика проведение служебный расследование с...
3999    условие работа крупный центр киберзащита стран...
4000    продукт защита периметр защита атака защита пр...
4001    предлагать оформление согласно трудовой кодекс...
Name: vacdesc_morphed, Length: 4002, dtype: object

In [83]:
vacs.to_csv('vacancy_morphed.csv', encoding='utf-8')

In [134]:
stop_russian = stopwords.words('russian')
text_transformer = TfidfVectorizer(stop_words=stop_russian, ngram_range=(1, 1), lowercase=True, max_features=15000)
text = text_transformer.fit_transform(vacs['vacdesc_morphed'])

## 3.Взять любой текст, описывающий опыт из резюме и аналогично предобработать.

In [216]:
CV = open('CV_description_simplified.txt', 'r', encoding = 'utf-8')

In [217]:
cv_exp = CV.read()

In [218]:
cv_exp

'• Анализ данных, составление отчётности, графиков, диаграмм, презентаций;\n• Работа с большими массивами данных в Excel и Access, использование функций массивов, ВПР, сводных таблиц, работа с макросами (VBA, на уровне чтения и частичной корректировки);\n• Разработка и согласование Планов развития и инвестиций с использованием методов регрессионного анализа.\n• Руководство командой аналитиков\n• Написание функциональных требований и согласование технических заданий к внутренним информационным системам\n• Работа с BI-аналитикой (определение исходных данных, проектирование структуры отчётов)\n'

In [219]:
cv_exp_cleaned = re.sub(r'[^А-Яа-яЁё ]+', ' ', str(cv_exp))

In [220]:
def lemmatize(row):
    t = []
    text = row
    for word in text.split():
        if len(word)<=3:
            continue
        p = morph.parse(word)[0]
        t.append(p.normal_form)
    return " ".join(t)
cv_morphed = lemmatize(cv_exp_cleaned)

In [221]:
cv_morphed

'анализ данные составление отчётность график диаграмма презентация работа больший массив данные использование функция массив сводный таблица работа макрос уровень чтение частичный корректировка разработка согласование план развитие инвестиция использование метод регрессионный анализ руководство команда аналитик написание функциональный требование согласование технический задание внутренний информационный система работа аналитика определение исходный данные проектирование структура отчёт'

In [222]:
cv_ready_tf_idf = pd.DataFrame([cv_morphed])
cv_ready_tf_idf.columns = ['a']

## 4. Посчитать векторное представление описаний вакансий и опыта из резюме.
## 5. Посчитать косинусную меру сходства для опыта и описаний вакансий
## 6. Вывести Топ-10 наиболее релевантных вакансий

In [223]:
text #вектор вакансий, из пункта 2, просто для памяти

<4002x12091 sparse matrix of type '<class 'numpy.float64'>'
	with 448992 stored elements in Compressed Sparse Row format>

In [224]:
text_cv = text_transformer.transform(cv_ready_tf_idf['a'])

In [225]:
text_cv

<1x12091 sparse matrix of type '<class 'numpy.float64'>'
	with 43 stored elements in Compressed Sparse Row format>

In [226]:
from sklearn.metrics.pairwise import linear_kernel

In [227]:
cosine_similarities = linear_kernel(text_cv, text).flatten()

In [228]:
vacs['cos_similarities'] = cosine_similarities
# и отсортируем по убыванию (т.к. cos(0)=1)
vacs = vacs.sort_values(by=['cos_similarities'], ascending=[0])

In [229]:
# Выведем 10 самых близких результатов
for index, row in vacs[0:10].iterrows():
    print(row['vactitle'], row['cos_similarities'])

Практикант 0.31606967550200676
Старший эксперт по технологиям 0.27730824460857545
Администратор проекта/delivery-менеджер 0.27534279893690683
Senior QA инженер 0.26954310921685964
Консультант по работе с ПФР 0.26210848775198553
Менеджер по продажам 0.24351257369148158
DevOps Инженер 0.24175156143823462
Старший менеджер по обслуживанию 0.2379008527334487
Кассир 0.23789559691072196
Региональный менеджер (Супер-коуч) 0.23652843351749347


### Для полной версии резюме результат:

Старший специалист по взысканию задолженности 0.29592914800589193
Ведущий специалист по взысканию 0.28416383394748446
Старший менеджер по обслуживанию 0.2798215626064644
Практикант (г. Челябинск) 0.27439373829957875
Руководитель проектов (Кластер "Управление ценой и доходностью") 0.27057265234994626
Ведущий специалист по взысканию 0.2585420788919523
Специалист по прямым продажа 0.25748112929621153
Главный специалист по проблемным активам крупного и среднего бизнеса 0.2553456849490878
Инженер (DevOps) 0.2531196995932508
Специалист по прямым продажам 0.24942572925356413